In [1]:
from dotenv import dotenv_values
import os
from pydantic_settings import BaseSettings, SettingsConfigDict


notebook_dir = os.getcwd()
project_dir = os.path.join(notebook_dir, os.pardir)
dotenv_path = os.path.join(project_dir, ".env")


class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file=dotenv_path, env_file_encoding="utf-8", extra="ignore"
    )

    picovoice_access_key: str


settings = Settings()

In [2]:
import pvporcupine

try:
    porcupine = pvporcupine.create(
    access_key=settings.picovoice_access_key,
    keyword_paths=['../models/Эй-Конь_ru_mac_v3_0_0.ppn'],
    model_path='../models/porcupine_params_ru.pv'
    )
except pvporcupine.PorcupineInvalidArgumentError as e:
    print("One or more arguments provided to Porcupine is invalid: ", args)
    print(e)
    raise e
except pvporcupine.PorcupineActivationError as e:
    print("AccessKey activation error")
    raise e
except pvporcupine.PorcupineActivationLimitError as e:
    print("AccessKey '%s' has reached it's temporary device limit" % args.access_key)
    raise e
except pvporcupine.PorcupineActivationRefusedError as e:
    print("AccessKey '%s' refused" % args.access_key)
    raise e
except pvporcupine.PorcupineActivationThrottledError as e:
    print("AccessKey '%s' has been throttled" % args.access_key)
    raise e
except pvporcupine.PorcupineError as e:
    print("Failed to initialize Porcupine")
    raise e

In [3]:
from pvrecorder import PvRecorder
from datetime import datetime

for i, device in enumerate(PvRecorder.get_available_devices()):
    print('Device %d: %s' % (i, device))

recorder = PvRecorder(
    frame_length=porcupine.frame_length,
    device_index=0)
recorder.start()

print('Listening ... (press Ctrl+C to exit)')

try:
    while True:
        pcm = recorder.read()
        result = porcupine.process(pcm)

        if result >= 0:
            print('[%s] Detected %s' % (str(datetime.now()), 'Эй-Конь'))
except KeyboardInterrupt:
    print('Stopping ...')
finally:
    recorder.delete()
    porcupine.delete()

Device 0: MacBook Pro Microphone
Listening ... (press Ctrl+C to exit)
[2024-06-18 20:11:38.282020] Detected Эй-Конь
[2024-06-18 20:11:40.260893] Detected Эй-Конь
[2024-06-18 20:11:42.963163] Detected Эй-Конь
[2024-06-18 20:11:54.211911] Detected Эй-Конь
Stopping ...
